<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-09 10:46:36
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: 37.57 K (0.27%)
Current PnL: -12.83 L (-8.94%)
CY Booked + Current PnL: -5.87 L (-4.09%)
-------------------
Total profit:  5.00 L
Total loss:  -17.83 L
-------------------
Total Booked + Current PnL: 21.11 L (18.11%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.07%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.34 L (51.96%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 11.04%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-217.49,H-SC,98.96,84757.0,-10559.0,10595.0,3.17,-11.08,12.50,0.04,157.0,-1.00,0.62,67.17,OX40N,NTT,MISC
4,APOLLOHOSP,6904.43,8285.00,-17.18,H-LC,90.62,112822.0,9256.0,11451.0,-0.58,8.94,10.15,20.00,32.0,0.81,0.82,24.27,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-21.55,H-LC,73.96,108464.0,4152.0,13384.0,-1.12,3.98,12.34,16.81,27.0,0.31,0.79,20.67,X5K,ATH,METALS
71,SBIN,760.30,863.00,-11.51,M-LC,29.17,214849.0,13370.0,13836.0,-0.23,6.64,6.44,13.51,77.0,0.97,1.57,20.07,XY25,NTT,BANKS
81,TATAELXSI,7332.28,7332.00,-18.15,X-MC,9.38,86002.0,-16650.0,16650.0,-0.46,-16.22,19.36,-0.00,59.0,-1.00,0.63,30.73,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-32.52,M-MC,7.29,178992.0,-816.0,113230.0,-0.50,-0.45,63.26,62.52,173.0,-0.01,1.30,33.48,XR,NTT,BANKS
38,HEROMOTOCO,4311.81,6039.03,-6.19,H-MC,34.38,151004.0,91.0,60356.0,0.33,0.06,39.97,40.06,103.0,0.00,1.10,23.51,AR,ATH,AUTO
60,NATIONALUM,189.63,247.44,-50.15,H-MC,50.00,100159.0,-345.0,30979.0,-0.49,-0.34,30.93,30.49,97.0,-0.01,0.73,31.95,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,LALPATHLAB,2804.19,3545.00,-28.55,H-SC,72.92,164450.0,10220.0,30522.0,3.82,6.63,18.56,26.42,127.0,0.33,1.20,29.68,X40N,NTT,HEALTHCARE
94,WHIRLPOOL,1167.49,2270.00,-38.56,M-SC,59.38,202786.0,31165.0,130898.0,3.49,18.16,64.55,94.43,201.0,0.24,1.48,49.67,XR,NTT,DURABLES
59,MEDANTA,1064.65,1486.00,13.85,X-SC,80.21,242852.0,37375.0,43956.0,3.30,18.19,18.10,39.58,65.0,0.85,1.77,30.14,XY24,NTT,HEALTHCARE
92,VIPIND,488.80,489.00,-217.49,H-SC,98.96,84757.0,-10559.0,10595.0,3.17,-11.08,12.50,0.04,157.0,-1.00,0.62,67.17,OX40N,NTT,MISC
54,KPIGREEN,497.21,731.79,27.90,H-SC,51.04,130914.0,5617.0,53491.0,2.43,4.48,40.86,47.18,143.0,0.11,0.95,63.78,MH,ATH,POWER


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.00,-6.01,M-LC,6.25,163864.0,23024.0,21466.0,-4.08,16.35,13.10,31.59,88.0,1.07,1.19,42.34,XY24,NTT,BANKS
0,5PAISA,564.75,565.00,147.60,H-SC,15.62,133691.0,-53241.0,53329.0,-2.97,-28.48,39.89,0.04,149.0,-1.00,0.97,27.90,OX40N,NTT,FINANCE
90,VALIANTORG,512.64,838.00,-357.02,H-SC,17.71,109330.0,-23956.0,108554.0,-1.87,-17.97,99.29,63.47,151.0,-0.22,0.80,83.26,XR,NTT,CHEMICALS
68,SAIL,130.64,228.00,90.28,M-MC,33.33,228578.0,3616.0,164028.0,-1.71,1.61,71.76,74.53,189.0,0.02,1.67,32.71,XY24,BTT,STEEL
37,HCLTECH,1647.60,1972.35,15.52,H-LC,57.29,102437.0,1933.0,17875.0,-1.69,1.92,17.45,19.71,34.0,0.11,0.75,23.62,X40,ATH,IT


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-217.49,H-SC,98.96,84757.0,-10559.0,10595.0,3.17,-11.08,12.50,0.04,157.0,-1.00,0.62,67.17,OX40N,NTT,MISC
79,SYMPHONY,1306.42,1306.0,-28.99,M-SC,26.04,151226.0,-19915.0,19856.0,0.03,-11.64,13.13,-0.03,191.0,-1.00,1.10,8.69,OX40N,NTT,DURABLES
87,UJJIVANSFB,52.77,53.0,44.69,M-SC,18.75,125766.0,-16713.0,17331.0,-1.17,-11.73,13.78,0.44,247.0,-0.96,0.92,48.44,OX40N,NTT,BANKS
46,INDIGOPNTS,1407.73,1408.0,127.68,H-SC,92.71,149172.0,-25387.0,25419.0,0.22,-14.54,17.04,0.02,133.0,-1.00,1.09,29.35,OX40N,NTT,PAINTS
53,KANSAINER,299.63,340.0,-67.47,H-SC,46.88,226305.0,-43362.0,79705.0,0.60,-16.08,35.22,13.47,137.0,-0.54,1.65,13.27,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-18.70,X-LC,35.42,265106.0,-30828.0,124732.0,-0.22,-10.42,47.05,31.73,1.0,-0.25,1.93,6.11,X40,BTT,IT
48,INFY,1461.46,2275.00,-8.83,X-LC,71.88,291913.0,30312.0,115306.0,-0.48,11.59,39.50,55.67,2.0,0.26,2.13,18.40,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-13.47,X-LC,60.42,271174.0,827.0,33788.0,1.19,0.31,12.46,12.81,5.0,0.02,1.98,13.41,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.76,X-LC,67.71,270158.0,40925.0,26367.0,0.58,17.85,9.76,29.35,10.0,1.55,1.97,28.37,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-3.50,X-LC,21.88,81560.0,-28612.0,117683.0,-1.04,-25.97,144.29,80.85,11.0,-0.24,0.59,20.12,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3079.90,NaN,NaN,93.75,76998.0,42549.0,0.0,1.08,123.51,0.00,123.51,274.0,inf,0.56,17.74,AR,ATH,ELECTRICAL
28,DIGITIDE,188.38,237.72,NaN,NaN,28.12,50236.0,8981.0,1824.0,-1.52,21.77,3.63,26.19,272.0,4.92,0.37,18.27,XY24,ATH,IT


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,90.28,M-MC,33.33,228578.0,3616.0,164028.0,-1.71,1.61,71.76,74.53,189.0,0.02,1.67,32.71,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.79,27.90,H-SC,51.04,130914.0,5617.0,53491.0,2.43,4.48,40.86,47.18,143.0,0.11,0.95,63.78,MH,ATH,POWER
20,CAMS,3643.00,5250.99,3.65,H-SC,22.92,116480.0,14476.0,30553.0,0.93,14.19,26.23,44.14,126.0,0.47,0.85,32.74,X40N,ATH,MISC
95,WIPRO,243.46,420.00,-2.86,M-LC,87.50,166377.0,15432.0,94020.0,-0.48,10.22,56.51,72.51,71.0,0.16,1.21,14.61,XR,NTT,IT
15,BLUESTARCO,1646.70,2337.55,-3.43,H-SC,96.88,181150.0,16480.0,52606.0,0.08,10.01,29.04,41.95,124.0,0.31,1.32,18.22,X40N,ATH,AC


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3079.90,NaN,NaN,93.75,76998.0,42549.0,0.0,1.08,123.51,0.00,123.51,274.0,inf,0.56,17.74,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.56,M-SC,59.38,202786.0,31165.0,130898.0,3.49,18.16,64.55,94.43,201.0,0.24,1.48,49.67,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.86,M-LC,87.50,166377.0,15432.0,94020.0,-0.48,10.22,56.51,72.51,71.0,0.16,1.21,14.61,XR,NTT,IT
45,INDIAMART,2327.09,4911.36,-46.96,H-SC,81.25,138155.0,14819.0,122143.0,2.08,12.02,88.41,111.05,123.0,0.12,1.01,35.37,AR,ATH,MISC
54,KPIGREEN,497.21,731.79,27.90,H-SC,51.04,130914.0,5617.0,53491.0,2.43,4.48,40.86,47.18,143.0,0.11,0.95,63.78,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,85.86,NaN,NaN,55.21,18803.0,-30790.0,0.0,1.34,-62.09,0.00,-62.08,270.0,-inf,0.14,26.10,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,63.54,65088.0,-48461.0,88513.0,1.04,-42.68,135.99,35.27,265.0,-0.55,0.47,59.30,XR,NTT,HOTELS
32,ENRIN,1377.95,3079.90,NaN,NaN,93.75,76998.0,42549.0,0.0,1.08,123.51,0.00,123.51,274.0,inf,0.56,17.74,AR,ATH,ELECTRICAL
92,VIPIND,488.80,489.00,-217.49,H-SC,98.96,84757.0,-10559.0,10595.0,3.17,-11.08,12.50,0.04,157.0,-1.00,0.62,67.17,OX40N,NTT,MISC
58,MASFIN,326.60,399.50,-17.80,H-SC,66.67,93300.0,-4680.0,26553.0,-0.19,-4.78,28.46,22.32,145.0,-0.18,0.68,35.32,XR,ATH,FINANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-15.73,L-SC,100.00,169559.0,27717.0,28859.0,-1.15,19.54,17.02,39.88,262.0,0.96,1.24,71.83,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-217.49,H-SC,98.96,84757.0,-10559.0,10595.0,3.17,-11.08,12.50,0.04,157.0,-1.00,0.62,67.17,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,7.77,X-SC,97.92,100229.0,-734.0,95268.0,-1.06,-0.73,95.05,93.63,67.0,-0.01,0.73,49.29,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-3.43,H-SC,96.88,181150.0,16480.0,52606.0,0.08,10.01,29.04,41.95,124.0,0.31,1.32,18.22,X40N,ATH,AC
5,ASIANPAINT,2961.56,4250.00,-13.75,H-LC,95.83,171292.0,-30094.0,117712.0,1.38,-14.94,68.72,43.51,31.0,-0.26,1.25,19.24,X40,BTT,PAINTS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.95
1,25,41.98
2,50,70.64


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.53
LC    33.85
MC    21.55
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.96
X40      12.93
XY25     12.07
X40N     11.99
XR       10.01
AR        8.18
OX40N     8.15
MH        1.68
X5K       1.42
X200      1.40
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.55
M-SC    15.86
H-LC    14.11
X-LC    12.90
H-MC     9.60
X-MC     7.57
M-LC     5.80
X-SC     4.50
M-MC     4.03
L-SC     2.62
L-LC     1.04
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.28,0.25,33.09
IT,12.08,-9.85,63.84
FINANCE,8.50,-11.64,52.15
BANKS,7.61,-9.22,53.79
MISC,5.91,-18.93,74.70
PAINTS,5.72,-11.38,33.34
ELECTRICAL,5.43,1.13,45.40
HEALTHCARE,4.72,0.59,28.45
AUTO,4.28,-14.88,65.40


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2642696.0,26
XR,1083169.0,13
AR,1028606.0,10
X40,680519.0,10
X40N,518195.0,11
XY25,464548.0,8
OX40N,311815.0,11
SR,178124.0,2
MH,84470.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1969703.0,20
M-SC,1766359.0,21
H-LC,605426.0,12
X-LC,539127.0,8
H-MC,516934.0,8
M-MC,417041.0,3
X-MC,413372.0,6
X-SC,300552.0,5
M-LC,264747.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      885519.0      6
M-SC       XY24      807538.0      7
H-SC       AR        525058.0      3
M-SC       XR        332746.0      4
M-MC       XY24      303811.0      2
H-MC       XY24      293543.0      3
X-LC       X40       275914.0      3
H-SC       XR        216500.0      3
           X40N      199792.0      4
H-LC       X40N      192549.0      4
M-SC       AR        186634.0      2
           OX40N     166727.0      5
X-MC       X40       157262.0      2
M-LC       XY24      156891.0      2
H-LC       X40       153013.0      3
L-SC       XR        150391.0      2
X-SC       XR        122233.0      1
M-SC       XY25      120550.0      1
X-LC       AR        117683.0      1
M-MC       XR        113230.0      1
X-MC       XY25      102870.0      1
X-LC       XY25       98519.0      3
X-SC       SR         95268.0      1
M-LC       XR         94020.0      1
H-SC       OX40N      89343.0      3
X-MC       XY24       83368.0      1
M-SC       SR         82856.0      1
H-MC       XY25       81413.0      1
H-LC       AR         80762.0      1
           X5K        73067.0      2
M-SC       X40        69308.0      1
H-LC       X200       68648.0      1
H-MC       AR         60356.0      1
L-SC       AR         58113.0      1
L-MC       XR         54049.0      1
H-SC       MH         53491.0      1
X-MC       X40N       53222.0      1
L-LC       XY25       47360.0      1
X-LC       X40N       47011.0      1
X-SC       XY24       43956.0      1
           OX40N      39095.0      2
H-LC       XY24       37387.0      1
H-MC       MH         30979.0      1
L-SC       XY24       28859.0      1
H-MC       X40N       25621.0      1
           X40        25022.0      1
X-MC       OX40N      16650.0      1
M-LC       XY25       13836.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
